# Data exploration with DataFrames, cont.

<font color = '#ed865c' size = 4>**Make sure to press the play button to run the cell below: this will re-load the datasets and functions that we worked on during the last section.**</font>

In [ ]:
#@title
# make sure to run this cell to import the external files we need for today
# and load in the appropriate packages
!git clone https://github.com/ccbskillssem/pythonbootcamp.git

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

alleles = pd.read_table('/content/pythonbootcamp/day_4/alleles.tsv')
clinvar = pd.read_table('/content/pythonbootcamp/day_4/clinvar.tsv')

## Changing labels

<font color = '#ed865c' size = 4>**Make sure to run the cell above before you begin: this will re-load the datasets and functions that we worked on during the last section.**</font>

### Row labels

Currently, the row labels for `alleles` are numeric, which makes it easy to index rows just as we did with arrays. However, it's also possible to use row *names* as labels, just as we do with columns.

To do this, we can select an existing column of the DataFrame and turn it into the row Index. However, it's **extremely** important that we make sure the candidate column contains only unique values: we wouldn't want to try and index a row name, only to retrieve multiple rows with the same name!

Series have a useful attribute called `.is_unique`, which is `True` if all values in the Series are unique. Given that columns of DataFrames are stored as Series, we can easily check our candidate columns.

In [ ]:
# try it out:
# print the .is_unique attribute of each column
# hint: remember for loops? try using a loop with the column labels


Great – looks like there's a candidate column for our row labels. We can now change `allele`'s row labels using the `.set_index()` method.

In [ ]:
# in this case, we're opting to update the variable

alleles = alleles.set_index('ID')
alleles.head()

Excellent! As you can see, each row's `ID` value is now the row label. We can access each row using `.loc[]` and the corresponding `ID` label.

In [ ]:
alleles.loc['rs573303859']

Row names can be convenient, but also cumbersome: for example, now that we've assigned `ID` as our row labels, all of the row labels are strings: thus, we can no longer slice multiple rows with `.loc[]`.

> If you really want to slice multiple rows *and* retain string-type row labels, you will need to use `.iloc[]` instead of `.loc[]`. `.iloc[]` is used to access columns/rows/slices/cells in a `numpy` array-like manner. You can read more about it [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-position).

In [ ]:
# try it out:
# access the row labeled rs563299082
# and its chromosome, position, reference, and alternative values


If you end up wanting to revert back to the standard 0-index numeric row labels, you can use the `.reset_index()` method to convert the row names back to a regular column.

In [ ]:
# let's return alleles to its original state

alleles = alleles.reset_index()
alleles.head()

### Column labels
In the case that we want to replace column labels, we can use the `.rename()` method, which is the equivalent of `.replace()` for column labels. Just as we showed you before, the easiest way to do this is to provide a dictionary of target and substitute labels.

For example, let's consider `reference` and `alternative`, which are commonly shortened to `REF` and `ALT` in genomic file formats. Changing the column labels while maintaining the column content is a single-line operation:

In [ ]:
# rename reference and alternative to REF and ALT for all rows

alleles.rename({'reference': 'REF',
                 'alternative': 'ALT'}, axis = 'columns').head()

## Sorting by column(s)
There's no way to sugar-coat it: there was no intuitive way for us to sort arrays by specific columns in `numpy`.

With `pandas`, the `.sort_values()` method allows us to sort a DataFrame/Series with the following inputs:
* A single column name, or a list of column names
* The `ascending` parameter, which takes a single Boolean value or a list of Boolean values (corresponding to the list of column names).
  * `True` will sort by lowest to highest (ascending).
  * `False` will sort by highest to lowest (descending).

In [ ]:
# let's sort AFR_AF by highest to lowest frequency

alleles.sort_values("AFR_AF", ascending=False).head()

⏸ **Exercise 1** Try sorting `alleles` by *all three* of the following columns:
* `AFR_AF`, from highest to lowest.
* `AMR_AF`, from lowest to highest.
* `EAS_AF`, from highest to lowest.

In [ ]:
### write your code below ###


## Adding columns

We can easily create new columns by simply assigning a new column name and contents to a DataFrame.

In [ ]:
# let's sample some rows with .sample()
# using 2024 as our seed value

some_alleles = alleles.sample(n = 5, random_state = 2024)

In [ ]:
some_alleles['sample_col'] = 0 # a new column full of zeros
some_alleles

---
⏸ **Exercise 2 (No coding)** What happens when you change the seed value of a random sampler (like `.sample()`)? What happens if you don't have a seed value specified?

Hint: One of the morning exercises might help you answer this question.

---

Next, we can also assign new columns by explicitly providing a column label and a "definition" for the column.

In [ ]:
# let's reuse some_alleles
# creating a new column that contains the *reference* allele frequency
# this takes advantage of numpy broadcasting

some_alleles['reference_AF'] = 1 - some_alleles['AF']
some_alleles

Lastly, the `.map()` method allows us to **mutate** new columns based on values of existing columns. This is useful for adding categorical or Boolean columns without overwriting existing columns.

In [ ]:
# let's mutate a new column called filter_pass
# instead of replacing the values of filter

alleles['filter_pass'] = alleles['filter'].map({'PASS': True,
                                      'LowQ': False})
alleles.head()

## Dropping columns

We can use the `.drop()` method to drop columns as easily as we created them. This can be handy if you end up with too many columns during your exploration, or if you simply don't need some of the columns that came with your dataset.

In [ ]:
# getting rid of filter_pass
# removing it from all rows in the dataframe

alleles = alleles.drop('filter_pass', axis = 'columns')
alleles

## Grouping

We've now reached the operations that are *only* possible in `pandas`. **Grouped operations** partition the dataset into subsets based on the value of a given column or columns prior to operation.

There are two major steps in performing grouped operations:
1. **Splitting**: Splitting data into groups.
2. **Applying**: Applying a function across separate groups. (This is where vectorization is helpful!)

We can accomplish the first step using the `.group_by()` method, which takes one or more column labels.

In [ ]:
# this method doesn't show a visible output

alleles.groupby('filter')

In order to see the effect of splitting with `.groupby()`, we need to chain `.groupby()` with a function that we want applied to our groups.

For example, let's say that we want to know the average quality score of alleles that pass (or don't pass) filter.

In [ ]:
alleles.groupby('filter')['quality'].mean()

## Merging (review)

This morning we discussed merging two datasets: `alleles` and `clinvar`. Merging is a very useful operation that we'll be using in tomorrow's final mini-project.

Once again, merge `alleles` and `clinvar` so that we have a single DataFrame called `merged`, which contains only the alleles present in both of the datsets.

In [ ]:
# try it out:
# merge alleles and clinvar as you did earlier today


## Exporting

This is our final lesson in `pandas`: hooray!

We've learned a lot about how to parse and manipulate our DataFrames. At some point in the near future, you'll likely want to save the results of your operations.

The `.to_csv()` method allows us to easily export our DataFrame to a `.csv` file. As the name implies, it will save a DataFrame to a `.csv` file, given a string (file name).

Let's go ahead and cap off our lecture by exporting `merged` to a file called `merged.csv`.

In [ ]:
merged.to_csv('merged.csv')

If you open the `Files` menu (left panel, folder icon), you'll see that the `merged.csv` file is now available.
___
**CAUTION**: Files that you save while using Colab are not retained after you close the notebook, as they only exist in Colab's temporary **session storage**. If you generate files and wish to keep them, make sure to download your files (with the same three dots menu) before you exit Colab.
___

That's it for our introduction to `pandas`! We hope you're starting to get a feel for how immensely powerful `pandas` can be. (Perhaps too powerful for a single bootcamp lecture...)

* To learn more about DataFrame methods, click here for the DataFrame documentation.
* To learn more about Series methods, click here for the DataFrame documentation.
* To learn about other common `pandas` routines, refer to the official `pandas` cookbook [here](https://pandas.pydata.org/docs/user_guide/cookbook.html). (No pandas were harmed in the making of this cookbook.)


# << `Exercises` >>

⏸ **Exercise 3a**: Just like with arrays, we can use the `.isna()` method to detect missing values in a DataFrame. Use this to identify all missing values in `clinvar`.

In [ ]:
### write your code below ###


⏸ **Exercise 3b** Let's chain together some `pandas` methods! Which columns of `clinvar` are missing values?

*Hint*: The `.any()` method returns a `True` if any of the values in the Series or array are `True`.

In [ ]:
### write your code below ###


⏸ **Exercise 3c**: Find the total number of missing values in each column of `clinvar`.

*Hint*: Recall that `False` Booleans are considered equivalent to `0`, and `True` values are considered equivalent to `1`.

In [ ]:
### write your code below ###


⏸ **Exercise 3d**: Which rows in `clinvar` are incomplete? Filter `clinvar` to show these rows.

*Hint*: You can change the axis along which `.any()` operates.

In [ ]:
### write your code below ###


⏸ **Exercise 4a** We're now going to move to data cleaning. 

First, select rows in ```clinvar``` that have complete data (no missing values): save this filtered DataFrame to a variable called `clinvar_complete`. What are the dimensions of `clinvar_complete`?

In [ ]:
### write your code below ###


In [ ]:
# use this cell to check your work
clinvar_complete.isna().any()

⏸ **Exercise 4b**: It seems like a significant number of `clinvar` rows contain missing values. Calculate the mean, median, and maximum number of missing values for `clinvar` rows.

*Hint*: This exercise uses the same principle as Exercise 3c.

In [ ]:
### write your code below ###



⏸ **Exercise 4c**: The `.dropna()` method accomplishes what we did with Boolean filtering, but it allows for additional flexibility when filtering rows. Look up the documentation for `.dropna()`, then use it to select rows in `clinvar` that have no more than 2 NA values. Assign this DataFrame to `clinvar_partial`. 

In [ ]:
### write your code below ###


⏸ **Exercise 4d**: How many rows remain compared to `clinvar_complete` and the original `clinvar` DataFrame?

In [ ]:
### write your code below ###


⏸ **Challenge**: Create a new DataFrame called `pathogenic` that contains only pathogenic variants.

*Hint*: The `clinical_significance` column contains several different strings with the substring `'pathogenic'`. How can you take advantage of this to filter `clinvar`? If you're not sure, check the morning notebook.)

In [ ]:
### write your code below ###


Find the number of pathogenic variants for each of three genes: `BRCA1`, `PAH`, and `CFTR`.

In [ ]:
### write your code below ###


# [Optional] More about `pandas`

Are you excited about `pandas`? Like, really excited? Here's some extra info on techniques that may be useful to you in the future. (We're not including this in the main lecture, as there's already a ton to go through!)

We'll focus on the `merged` dataset, which we derived by merging together `alleles` and `clinvar`. All of the rows in `merged` represent alleles found in both datasets.

First, let's quickly inspect a couple of different things, just to get our bearings.

In [ ]:
# try it out:
# print the # of rows and cols in merged
# just to review how many alleles in alleles are also found in clinvar


In [ ]:
# try it out:
# view the first 10 rows of merged


Let's quickly look at the column labels, just so we can have a list of the columns we might want to examine.

In [ ]:
# try it out:
# examine the column labels: how many are there, and what are they?


Great! We can see that we have a good number of columns that derive from `clinvar`. There are also some duplicate columns, which are marked with the `_x` suffix to indicate that they derive from `alleles`, or the `_y` suffix to indicate that they derive from `clinvar`. You can adjust these suffixes using parameters in `pd.merge()`.

Let's examine some of the columns that may hold interesting categorical values:

In [ ]:
# try it out:
# find the unique values in the clinical_significance column


In [ ]:
# try it out:
# find the unique values in the molecular_consequence column


Given that not everyone will be versed in disease genetics or molecular biology, here's a quick crash course on what these mean.

*clinical_significance* refers to the observed effect of the mutation on human health. This data is curated by clinicians and uploaded to the ClinVar database.

*molecular_consequence* refers to the consequence of a specific mutation on the gene product (protein) of the mutated gene. Not all mutations yield the same molecular_consequence, and some consequences are more dire than others: for example, *nonsense mutations* can result in a gene product that may not perform its biological function.

## Fun tricks: rolling window operations

This is a fun feature of `pandas` that may be interesting to people who work with longitudinal-type data, such as time series data or genomic coordinate-related data. It is *not* a must-know feature of `pandas`.

The `.rolling()` method for DataFrames takes a `window` parameter, which specifies the number of rows to use to create a *rolling (sliding) window* across the DataFrame. This can be used to methodically calculate summaries such as sliding window averages using a `for` loop. (Yes, this is a very specific trick!)

In [ ]:
# calculating a rolling window average of allele frequencies on the 21st chromosome
# using the alleles dataset, sorted by genomic position on the 21st chromosome

rolling_mean = []
for window in alleles[alleles['chromosome'] == 21].sort_values('position').rolling(window = 500):
  rolling_mean.append(window['AF'].mean()) # get the average allele freq in the rolling wondow

In [ ]:
# plotting the rolling mean of allele frequencies across chromosome 21
# plt.plot takes two arrays/lists: the first corresponds to x values, the second to y values
# below, we created "dummy" x values that match the length of rolling_mean
# (there are more robust ways to plot this rolling mean :) )

plt.plot(np.arange(0, len(rolling_mean)), rolling_mean)

Want to learn more fun `pandas` tricks? It's time for you to move on to the full documentation [here](https://pandas.pydata.org/docs/user_guide/index.html#user-guide). Go wild!